# Solving Burgers with Transfer Learning

## Import Relevant Packages

In [1]:
import numpy as np
import sciann as sn
from tensorflow.keras import callbacks

---------------------- SCIANN 0.6.1.1 ---------------------- 
For details, check out our review paper and the documentation at: 
 +  "https://arxiv.org/abs/2005.08803", 
 +  "https://www.sciann.com". 

 Need support or would like to contribute, please join sciann`s slack group: 
 +  "https://join.slack.com/t/sciann/shared_invite/zt-ne1f5jlx-k_dY8RGo3ZreDXwz0f~CeA" 
 


## Get Inputs and Neural Network Approximations

In [2]:
# Define the necessary inputs using Variable 
x = sn.Variable('x') # space
t = sn.Variable('t') # time
d = sn.Variable('d') # boundary perturbation

v_low = 0.05 # viscosity for low fidelity model
v_high = .001 # viscosity for high fidelity model

# Get neural network predictions for u for the low fidelity model
u_low = sn.Functional('u_low', [d,t,x], 8*[20], 'tanh') 

## Set up Optimization Problem with Boundary Conditions

In [3]:
from sciann.utils.math import diff, sign, sin

TOL = 0.001
# Low fidelity partial differential equation
L1 = diff(u_low, t) - v_low * diff(u_low, x, order=2)

# Low fidelity boundary conditions
L2 = (1 + sign(0-t)) * (u_low - 1 + (1+x)*(1 + d/2))
L3 = (1 + sign(-1-x)) * u_low * (1 + d)
L4 = (1 + sign(x-1)) * u_low

## Build PINN

In [4]:
m = sn.SciModel([x,t,d], [L1, L2, L3, L4])

## Get Training Data

In [5]:
x_data, t_data, d_data = np.meshgrid(
    np.linspace(-1, 1, 10),
    np.linspace(0, 12, 10),
    np.linspace(0, 0.1, 10))

## Train PINN

In [6]:
h = m.train(
    [x_data, t_data, d_data], 
    4*['zero'], 
    learning_rate=0.001, 
    epochs=10000,
    callbacks = [callbacks.EarlyStopping(patience = 0)],
    verbose = 0)


Total samples: 1000 
Batch size: 64 
Total batches: 16 



# Transfer Learning

## Get 'u' and Set up Optimization Problem and Boundary Conditions for High Fidelity

In [7]:
# Get neural network predictions for u for the high fidelity model
u_high = sn.Functional('u_high', [d,t,x], 8*[20], 'tanh')

# High fidelity partial differential equation
L5 = diff(u_high, t) + u_high * diff(u_high, x) - v_high * diff(u_high, x, order=2)

# High fidelity boundary conditions (same as low fidelity model)
L6 = (1 + sign(0-t)) * (u_high - 1 + (1+x)*(1 + d/2))
L7 = (1 + sign(-1-x)) * u_high * (1 + d)
L8 = (1 + sign(x-1)) * u_high

## Get Information Learned from the Low Fidelity Model to use in the new PINN

In [8]:
# Set the weights used to approximate the low fidelity u to the weights of the neural network 
# that will approximate the high fidelity u
u_high.set_weights(u_low.get_weights())

# Prevent all they layers except the last one from being retrained so the low fidelity model information is not
# lost as well as to prevent overfitting
u_high.set_trainable(False, [1,2,3,4,5,6,7])

## Build the New Multi-Fidelity PINN

In [9]:
m2 = sn.SciModel([x,t,d], [L5, L6, L7, L8])

## Train PINN

In [10]:
# Implement an early stopping callback to prevent the model from training when loss increases
h2 = m2.train(
    [x_data, t_data, d_data], 
    4*['zero'], 
    learning_rate=0.001, 
    epochs=1000,
    callbacks = [callbacks.EarlyStopping(patience = 1)],
    verbose = 0)


Total samples: 1000 
Batch size: 64 
Total batches: 16 



## Get Predictions

In [11]:
u_pred = u_high.eval(m2, [x_data, t_data, d_data])